In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from openai import OpenAI

# --- 설정 ---
API_KEY = "" 
INPUT_CSV = "hard_samples_for_llm.csv" # 1000개 후보군 파일
OUTPUT_CSV = "gold_labels2.csv" # 이번에 저장할 950개 파일

# 이미 0~49번(50개)은 파일럿으로 했으니, 50번부터 시작합니다.
START_INDEX = 50   
COUNT = 950        # 50+950 = 1000

client = OpenAI(api_key=API_KEY)

# --- LLM 함수 ---
def get_gold_labels(text, candidates):
    prompt = f"""
    You are an expert classifier for Amazon products.
    Task: Select exactly 2 or 3 most relevant category IDs from the Candidate List.
    Review: "{text}"
    Candidate List:
    {candidates}
    Output Format:
    ID, ID, ID (No other text)
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return None

# --- 실행 ---
if not os.path.exists(INPUT_CSV):
    print("❌ Error: hard_samples_for_llm.csv 파일이 없습니다.")
else:
    df = pd.read_csv(INPUT_CSV)
    
    # [중요] 전체 1000개 중 50번째부터 950번째까지만 자름
    target_df = df.iloc[START_INDEX : START_INDEX + COUNT]
    
    print(f"🚀 Starting Main Run: Index {START_INDEX} ~ {START_INDEX + COUNT - 1}")
    print(f"📊 Target Samples: {len(target_df)} (Expected: 900)")
    
    results = []
    for idx, row in tqdm(target_df.iterrows(), total=len(target_df)):
        pid = row['pid']
        text = row['text']
        candidates = row['candidates'].replace(" | ", "\n")
        
        labels = get_gold_labels(text, candidates)
        
        if labels:
            clean_labels = labels.replace(".", "").strip()
            results.append([pid, clean_labels])
            
        # 100개마다 중간 저장 (안전장치)
        if len(results) % 100 == 0:
            pd.DataFrame(results, columns=['id', 'labels']).to_csv(OUTPUT_CSV, index=False)

    # 최종 저장
    final_df = pd.DataFrame(results, columns=['id', 'labels'])
    final_df.to_csv(OUTPUT_CSV, index=False)
    